## (1)
### Import libraries

In [1]:
import time
import numpy as np
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori

### Input trans.txt and transform it into Dataframe

In [2]:
lines = open('trans.txt','r')
data = [line[:-1].split("\t") for line in lines]
te = TransactionEncoder()
te_array = te.fit(data).transform(data)
df = pd.DataFrame(te_array, columns=te.columns_)
df

,0,1,10,100,1000,1001,1002,1003,1004,1005,...,990,991,992,993,994,995,996,997,998,999
0,True,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,True,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
668328,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
668329,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
668330,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
668331,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


### Perform apriori algorithm

In [168]:
start_time = time.time()
Itemsets = apriori(df, min_support=0.0001, use_colnames=True, verbose=1, low_memory=True)
print("Running time: %s seconds" % (round(time.time() - start_time,2)))

Processing 540 combinations | Sampling itemset size 543
Running time: 202.31 seconds


### Preview Itemsets

In [80]:
Itemsets

,support,itemsets
0,0.073112,(0)
1,0.150766,(1)
2,0.000738,(10)
3,0.004110,(100)
4,0.009675,(101)
...,...,...
18689,0.000133,"(30, 143, 15, 24, 52)"
18690,0.000189,"(30, 15, 95, 24, 143)"
18691,0.000102,"(30, 15, 95, 24, 17)"
18692,0.000102,"(15, 95, 24, 17, 50)"


### Create column k and minFreq then output the results

In [7]:
arr=[0.0005,0.0004,0.0003,0.0002,0.0001]
def min_freq(sup):
    for i in arr:
        if sup>i:
            return i
result = Itemsets.copy()
result['k'] = [len(l) for l in result['itemsets']]
result['minFreq'] = [min_freq(i) for i in result['support']]
result = result.groupby(['k','minFreq']).size().unstack().fillna(0)
for i,j in enumerate(arr):
    if j!=0.0005:
        result[j]+=result[arr[i-1]]
result

NameError: name 'Itemsets' is not defined

## (2)
### Accelerating by picking random samples
##### using O ( (1/ε^2)*(D + log 1/δ)) for ε = 1e-2,δ = 0.5%, D = 7


In [3]:
import math
start_time = time.time()
np.random.seed(56173366)# for reproducible results.
split = len(df)//int((1/0.01**2)*(7 + math.log(1/0.005,10)))
dfs = np.array_split(df.reindex(np.random.permutation(df.index)),split)[0]
sample_Itemsets = apriori(dfs, min_support=0.0001, use_colnames=True, verbose=1, low_memory=True)
sample_Itemsets
print("Running time: %s seconds" % (round(time.time() - start_time,2)))

Processing 7 combinations | Sampling itemset size 6 5 4
Running time: 20.93 seconds


### Preview sample_Itemsets

In [188]:
sample_Itemsets

,support,itemsets
0,0.073924,(0)
1,0.150759,(1)
2,0.000775,(10)
3,0.004158,(100)
4,0.009458,(101)
...,...,...
20009,0.000105,"(15, 95, 24, 17, 53)"
20010,0.000105,"(15, 95, 6, 24, 17)"
20011,0.000147,"(30, 15, 95, 24, 53)"
20012,0.000105,"(804, 768, 763, 772, 775)"


### Create column k and minFreq then output the results

In [196]:
result2 = sample_Itemsets.copy()
result2['k'] = [len(l) for l in result2['itemsets']]
result2['minFreq'] = [min_freq(i) for i in result2['support']]
result2 = result2.groupby(['k','minFreq']).size().unstack().fillna(0)
for i,j in enumerate(arr):
    if j!=0.0005:
        result2[j]+=result2[arr[i-1]]
result2

minFreq,0.0001,0.0002,0.0003,0.0004,0.0005
k,,,,,
1,726.0,597.0,524.0,473.0,438.0
2,8251.0,4516.0,3175.0,2258.0,1759.0
3,8725.0,2841.0,1461.0,829.0,536.0
4,2166.0,395.0,148.0,61.0,31.0
5,145.0,8.0,1.0,0.0,0.0
6,1.0,0.0,0.0,0.0,0.0


Although the result has a little different, but the running time reduced from 202sec to 15sec on my computer.

### Accelerating by partitioning

In [4]:
#divide into 5 parts
start_time = time.time()
dfp1 = df[:133666]
dfp2 = df[133666:133666*2]
dfp3 = df[133666*2:133666*3]
dfp4 = df[133666*3:133666*4]
dfp5 = df[133666*4:]

Itemsets1 = apriori(dfp1, min_support=0.0001, use_colnames=True, verbose=1, low_memory=True)
Itemsets1['support']*=len(dfp1)
Itemsets2 = apriori(dfp2, min_support=0.0001, use_colnames=True, verbose=1, low_memory=True)
Itemsets2['support']*=len(dfp2)
Itemsets3 = apriori(dfp3, min_support=0.0001, use_colnames=True, verbose=1, low_memory=True)
Itemsets3['support']*=len(dfp3)
Itemsets4 = apriori(dfp4, min_support=0.0001, use_colnames=True, verbose=1, low_memory=True)
Itemsets4['support']*=len(dfp4)
Itemsets5 = apriori(dfp5, min_support=0.0001, use_colnames=True, verbose=1, low_memory=True)
Itemsets5['support']*=len(dfp5)
Parti_Itemsets = pd.concat([Itemsets1, Itemsets2, Itemsets3, Itemsets4, Itemsets5])
Parti_Itemsets = Parti_Itemsets.groupby('itemsets')['support'].sum()
Parti_Itemsets[:] /= len(df)
print("Running time: %s seconds" % (round(time.time() - start_time,2)))
Parti_Itemsets = Parti_Itemsets.drop(Parti_Itemsets[Parti_Itemsets[:] < 0.0001].index)

Processing 98 combinations | Sampling itemset size 6 54
Processing 161 combinations | Sampling itemset size 654
Processing 14 combinations | Sampling itemset size 6 54
Processing 918 combinations | Sampling itemset size 5 4
Processing 492 combinations | Sampling itemset size 543
Running time: 274.65 seconds


### Create column k and minFreq then output the results

In [8]:
result3 = pd.DataFrame(Parti_Itemsets.copy())
result3['k'] = [len(l) for l in result3.index]
result3['minFreq'] = [min_freq(float(i)) for i in result3['support']]
result3
result3 = result3.groupby(['k','minFreq']).size().unstack().fillna(0)
for i,j in enumerate(arr):
    if j!=0.0005:
        result3[j]+=result3[arr[i-1]]
result3


minFreq,0.0001,0.0002,0.0003,0.0004,0.0005
k,,,,,
1,689.0,586.0,518.0,469.0,432.0
2,7060.0,4356.0,2977.0,2190.0,1703.0
3,6156.0,2573.0,1316.0,781.0,491.0
4,1153.0,307.0,125.0,48.0,26.0
5,51.0,4.0,1.0,0.0,0.0
